In [22]:
import ase
import ase.build
import numpy as np
from openbabel import openbabel as ob
import os
import pickle
import pandas as pd
import sys
from typing import Tuple

sys.path.append('..')
sys.path.append('../analyses')
import qm9

In [5]:
### from analysis.py
def construct_molecule(molecule_str: str) -> Tuple[ase.Atoms, str]:
    """Returns a molecule from the given input string.

    The input is interpreted either as an index for the QM9 dataset,
    a name for ase.build.molecule(),
    or a file with atomic numbers and coordinates for ase.io.read().
    """
    # If we believe the string is a file, try to read it.
    if os.path.exists(molecule_str):
        filename = os.path.basename(molecule_str).split(".")[0]
        return ase.io.read(molecule_str), filename

    # A number is interpreted as a QM9 molecule index.
    if molecule_str.isdigit():
        dataset = qm9.load_qm9("qm9_data")
        molecule = dataset[int(molecule_str)]
        return molecule, f"qm9_index={molecule_str}"

    # If the string is a valid molecule name, try to build it.
    molecule = ase.build.molecule(molecule_str)
    return molecule, molecule.get_chemical_formula()

In [13]:
methane = construct_molecule('CH4')[0]

In [15]:
obmol = ob.OBMol()
obmol.BeginModify()
for p, n in zip(methane.positions, methane.numbers):
    obatom = obmol.NewAtom()
    obatom.SetAtomicNum(int(n))
    obatom.SetVector(*p.tolist())
# infer bonds and bond order
obmol.ConnectTheDots()
obmol.PerceiveBondOrders()
obmol.EndModify()

In [23]:
ob.OBFingerprint.GetFingerprint(obmol)

TypeError: Wrong number or type of arguments for overloaded function 'OBFingerprint_GetFingerprint'.
  Possible C/C++ prototypes are:
    OpenBabel::OBFingerprint::GetFingerprint(OpenBabel::OBBase *,std::vector< unsigned int,std::allocator< unsigned int > > &,int)
    OpenBabel::OBFingerprint::GetFingerprint(OpenBabel::OBBase *,std::vector< unsigned int,std::allocator< unsigned int > > &)
